In [2]:
import cv2
import os
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('yolov8s.pt')

# Input video
video_path = 'input_video4.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

# Output video setup (same directory)
output_path = os.path.join(os.path.dirname(video_path), 'output_detected_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (
    int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
))

# COCO class IDs
PEDESTRIAN_ID = 0
CAR_ID = 2
ZEBRA_CROSSING_ID = 9  # Assumed if your model is custom-trained or supports it

# Previous positions for speed estimation
previous_car_positions = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    current_car_positions = {}

    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        center = ((x1 + x2) // 2, (y1 + y2) // 2)

        if cls_id == PEDESTRIAN_ID:
            label = f"Person {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        elif cls_id == CAR_ID:
            speed = 0
            for prev_center in previous_car_positions:
                dist = np.linalg.norm(np.array(center) - np.array(prev_center))
                if dist < 50:  # Match car
                    speed = dist * fps * 0.036  # Approx. speed in km/h
                    break
            label = f"Car {speed:.1f} km/h"
            current_car_positions[center] = (x1, y1, x2, y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 100, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)

        elif cls_id == ZEBRA_CROSSING_ID:
            label = f"Zebra {conf:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    previous_car_positions = current_car_positions
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Video saved to: {output_path}")



0: 384x640 19 persons, 1 bicycle, 8 cars, 1 motorcycle, 2 buss, 1 truck, 1 traffic light, 175.5ms
Speed: 4.2ms preprocess, 175.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 persons, 7 cars, 2 buss, 1 truck, 1 traffic light, 162.2ms
Speed: 1.8ms preprocess, 162.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 persons, 7 cars, 2 buss, 1 truck, 1 traffic light, 1 suitcase, 158.5ms
Speed: 2.3ms preprocess, 158.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 8 cars, 2 buss, 1 truck, 1 traffic light, 1 suitcase, 161.9ms
Speed: 1.5ms preprocess, 161.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 persons, 7 cars, 2 buss, 1 truck, 1 traffic light, 157.3ms
Speed: 1.6ms preprocess, 157.3ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 persons, 7 cars, 1 truck, 1 traffic light, 155.7ms
Speed: 1.9ms preprocess, 

In [17]:
import cv2
from ultralytics import YOLO

# Load the pre-trained YOLOv8 model (yolov8s.pt)
model = YOLO('yolov8s.pt')  # Adjust path if necessary

# Input and output video file paths
input_video = '3.mp4'
output_video = 'output3.mp4'

# Open the input video
cap = cv2.VideoCapture(input_video)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up the video writer to save output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID' for .avi output
out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)

    # Process detection results
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls)  # Class ID for the detected object
            label = model.names[class_id]  # Object label (e.g., 'car', 'fire truck', 'parking meter')

            # Print the detected object label to check what the model recognizes
            print("Detected object:", label)

            # Check if the label corresponds to a fire truck or a parking meter
            if label == 'fire truck':  # Label for fire truck
                # Draw bounding box and label on the frame for fire truck
                x1, y1, x2, y2 = map(int, box.xyxy)  # Coordinates for the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green bounding box
                cv2.putText(frame, 'Fire Truck', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            elif label == 'parking meter':  # Label for parking meter
                # Draw bounding box and label on the frame for parking meter
                x1, y1, x2, y2 = map(int, box.xyxy)  # Coordinates for the bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue bounding box for parking meter
                cv2.putText(frame, 'Parking Meter', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Write the processed frame to the output video
    out.write(frame)

# Release video resources
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 6 persons, 32 cars, 5 motorcycles, 1 truck, 178.2ms
Speed: 3.2ms preprocess, 178.2ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: motorcycle
Detected object: person
Detected object: motorcycle
Detected object: motorcycle
Detected object: car
Detected object: car
Detected object: person
Detected object: person
Detected object: person
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: car
Detected object: motorcycle
Detected object: motorcycle
Detected object: car
Detected object: car
Detected object: car
Detected object: person
Detected object: car
Detected object: car
Detected ob